# 背景

利用均线特征中的强势股特征进行选股。比如合力科技在2021年11月2日收盘时满足：

| 多头(30 60 120)    | 10              |
|------------------|-----------------|
| 5x10             | 6日前死叉           |
| 10x20            | 2日前死叉           |
| 支撑线              | 20日均线           |
| 支撑位              | 0.1%            |
| 趋势线              | 沿30日线上行         |
| 一阳穿多线            | 5 10 20日均线      |


此后开启一波凌历上涨，13个交易日翻倍。此外，它还有以下特征：

1. 7月8日连续2日放量
2. 9月23日连续两日放量
3. 上述两日均出现涨停

In [1]:
from alpha.notebook import *
from alpha.core.rsi_stats import rsi30, rsiday
await init_notebook()

In [5]:
def features(code, name, bars, results, frame_type):
    try:
        mafeatures = MaLineFeatures(bars)
    except Exception:
        return
    
    vec = mafeatures.as_vector()
    
    if frame_type == FrameType.MIN30:
        stats = rsi30
    else:
        stats = rsiday
        
    rsi = relative_strength_index(bars["close"][-30:])[-1]
    prob = stats.get_proba(code, rsi)
    
    results.append((name, code, *vec, rsi, prob))
    
def features_and_label(code, name, bars, results, frame_type):
    """
    从后10个bar中提取label
    """
    print(len(bars))
    xbars = bars[:-10]
    ybars = bars[-10:]
    xclose = xbars["close"]
    yclose = ybars["close"]
    
    pmax = np.argmax(yclose)
    pmin = np.argmin(yclose)

    if pmax == 0: # 单边下跌
        y = False


    if pmin < pmax and yclose[pmin]/xclose[-1] - 1 <= -0.05:
        y = False

    y = (yclose[pmax]/xclose[-1] - 1) > 0.1
    
    mafeatures = MaLineFeatures(xbars)
    
    vec = mafeatures.as_vector()
    
    if frame_type == FrameType.MIN30:
        stats = rsi30
    else:
        stats = rsiday
        
    # RSI反转特征
    rsi = relative_strength_index(xclose[-30:])[-1]
    prob = stats.get_proba(code, rsi)
    
    # 成交量特征
    vol_features = top_volume_direction(xbars)
    
    results.append((name, code, xbars[-1]["frame"], *vec, *vol_features, rsi, prob, y))

In [12]:
results = await scan(features_and_label, 100, nstocks=10, silent=False)
len(results)

2021-11-21 15:53:56,084 E 8207 alpha.notebook:scan:255 | not enough bars, 129 required, 68 passed
Traceback (most recent call last):
  File "/Users/aaronyang/miniforge3/envs/alpha/lib/python3.9/site-packages/alpha/notebook/__init__.py", line 252, in scan
    trigger(code, name, bars, results, frame_type)
  File "/var/folders/b5/73vzvtdn4pn_8wt6rpd2tb_w0000gn/T/ipykernel_8207/792456949.py", line 41, in features_and_label
    mafeatures = MaLineFeatures(xbars)
  File "/Users/aaronyang/miniforge3/envs/alpha/lib/python3.9/site-packages/alpha/features/maline.py", line 30, in __init__
    raise ValueError(f"not enough bars, {min_bars} required, {len(bars)} passed")
ValueError: not enough bars, 129 required, 68 passed
2021-11-21 15:53:56,090 E 8207 alpha.notebook:scan:255 | not enough bars, 129 required, 68 passed
Traceback (most recent call last):
  File "/Users/aaronyang/miniforge3/envs/alpha/lib/python3.9/site-packages/alpha/notebook/__init__.py", line 252, in scan
    trigger(code, name, 

78
78
78
78
78
78
78
78
78


0

In [ ]:
columns = ["name", "code","frame", *mf.feature_names, "v1", "v2", "rsi", "prsi", "y"]
df = pd.DataFrame(results, columns=columns)

In [ ]:
pd.options.display.max_rows = 500
df[(df["parallel(30,60,120)"]>0)&(df.bull_strike>2) &(df.supress.isna()) & (df.trendline==30)]

In [21]:
day_bars = await get_bars('603917.XSHG', 160, '1d', end = '2021-11-03')
mf = MaLineFeatures(day_bars)
len(mf.feature_names)

15

In [13]:
zgba = await get_bars("000009.XSHE", 180, "1d")

In [ ]:
results = []
features_and_label("000009.XSHE", "中国宝安", zgba, results, FrameType.DAY)

array([datetime.date(2021, 9, 1), datetime.date(2021, 9, 2),
       datetime.date(2021, 9, 3), datetime.date(2021, 9, 6),
       datetime.date(2021, 9, 7), datetime.date(2021, 9, 8),
       datetime.date(2021, 9, 9), datetime.date(2021, 9, 10),
       datetime.date(2021, 9, 13), datetime.date(2021, 9, 14),
       datetime.date(2021, 9, 15), datetime.date(2021, 9, 16),
       datetime.date(2021, 9, 17), datetime.date(2021, 9, 22),
       datetime.date(2021, 9, 23), datetime.date(2021, 9, 24),
       datetime.date(2021, 9, 27), datetime.date(2021, 9, 28),
       datetime.date(2021, 9, 29), datetime.date(2021, 9, 30),
       datetime.date(2021, 10, 8), datetime.date(2021, 10, 11),
       datetime.date(2021, 10, 12), datetime.date(2021, 10, 13),
       datetime.date(2021, 10, 14), datetime.date(2021, 10, 15),
       datetime.date(2021, 10, 18), datetime.date(2021, 10, 19),
       datetime.date(2021, 10, 20), datetime.date(2021, 10, 21),
       datetime.date(2021, 10, 22), datetime.date(202